**0. Imports**

In [75]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import random
import time
from fake_useragent import UserAgent
import requests
import json
import urllib.request, urllib.error, urllib.parse
from bs4 import BeautifulSoup
from bs4 import SoupStrainer
import re

**1. Prepare URLs**

In [3]:
netflix = pd.read_csv("netflix_dataset.csv", encoding="unicode-escape")
netflix = netflix.iloc[:,[1,2,7]]

netflix["api_url"] = "https://www.rottentomatoes.com/api/private/v2.0/search?q="+netflix["title"].replace(" ","+", regex=True).replace("&","and",regex=True)

**A. Headers List**

In [9]:
headers_list = [
    # Chrome 107.0 Windows 10
    {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 107.0 Mac
    {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate, br",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1"
    },
    # Chrome 107.0 iPadOS
    {
    "Connection": "keep-alive",
    "DNT": "1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (iPad; CPU OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/107.0.5304.66 Mobile/15E148 Safari/604.1",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-GB,en-US;q=0.9,en;q=0.8"
    },
    # Edge 107.0.1418 Win10
    {
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.28",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-User": "?1",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9"
    },

    # Edge 107.0.1418 Win10
    {
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36 Edg/107.0.1418.28",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-User": "?1",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9"
    },

    # Safari 605.1.15 Mac
    {
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 13_0) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.1 Safari/605.1.15",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-User": "?1",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9"
    },

    # Firefox 106.0 Mac
    {
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 13.0; rv:106.0) Gecko/20100101 Firefox/106.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-User": "?1",
    "Sec-Fetch-Dest": "document",
    "Referer": "https://www.google.com/",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9"
    }
]

**B. Function: Extract JSON / HTTP Response**

In [5]:
def get_api(url, flag):
    headers = random.choice(headers_list)
    r = requests.Session()
    r.headers = headers
    if flag:
        return json.loads(r.get(url).text)
    else:
        return r.get(url).text

**2. Extract movie/show URL from name using API**

In [239]:
main_url = []

for idx, row in netflix.iterrows():

    print(idx)

    try:
        dict = get_api(row["api_url"],1)
    
    except:
        main_url.append("Not found on RT")
        time.sleep(random.uniform(5,10))
        continue

    if row["type"] == "TV Show":
        if dict["tvCount"] != 0:
            index = 0
            for l in dict["tvSeries"]:
                if l["title"].lower() == row["title"].lower():
                    index = dict["tvSeries"].index(l)

            main_url.append(dict["tvSeries"][index]["url"])
        else:
            main_url.append("Not found on RT")
    
    if row["type"] == "Movie":
        if dict["movieCount"] != 0:
            index = 0
            for l in dict["movies"]:
                if l["name"].lower() == row["title"].lower():
                    index = dict["movies"].index(l)
            main_url.append(dict["movies"][index]["url"])
        else:
            main_url.append("Not found on RT")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

**3. Write the target_urls to file**

In [250]:
netflix["target_url"] = main_url
netflix.loc[netflix["target_url"] != "Not found on RT", "target_url"] = "https://www.rottentomatoes.com" + netflix["target_url"]
netflix.to_csv("intermediate op.csv")

**4. Read the target_urls from file**

In [7]:
netflix = pd.read_csv("intermediate op.csv", encoding="unicode-escape", index_col=0)
netflix

,type,title,release_year,api_url,target_url
0,Movie,Dick Johnson Is Dead,2020,https://www.rottentomatoes.com/api/private/v2....,https://www.rottentomatoes.com/m/dick_johnson_...
1,TV Show,Blood & Water,2021,https://www.rottentomatoes.com/api/private/v2....,https://www.rottentomatoes.com/tv/blood_and_wa...
2,TV Show,Ganglands,2021,https://www.rottentomatoes.com/api/private/v2....,https://www.rottentomatoes.com/tv/ganglands/s01
3,TV Show,Jailbirds New Orleans,2021,https://www.rottentomatoes.com/api/private/v2....,https://www.rottentomatoes.com/tv/jailbirds_ne...
4,TV Show,Kota Factory,2021,https://www.rottentomatoes.com/api/private/v2....,https://www.rottentomatoes.com/tv/kota_factory
...,...,...,...,...,...
8802,Movie,Zodiac,2007,https://www.rottentomatoes.com/api/private/v2....,https://www.rottentomatoes.com/m/zodiac_2014
8803,TV Show,Zombie Dumb,2018,https://www.rottentomatoes.com/api/private/v2....,Not found on RT
8804,Movie,Zombieland,2009,https://www.rottentomatoes.com/api/private/v2....,https://www.rottentomatoes.com/m/zombieland
8805,Movie,Zoom,2006,https://www.rottentomatoes.com/api/private/v2....,https://www.rottentomatoes.com/m/zoom_2016


**5. Get Critic and Audience Rating**

In [51]:
genre = []
tomato_score = []
audience_score = []

genre_count = 0

for idx, row in netflix.iterrows():
    
    print(idx, row["target_url"])

    if not (row["target_url"] == "Not found on RT"):
        try:
            html = get_api(row["target_url"],0)
        
        except Exception as e:
            print(e)
            genre.append("Error")
            tomato_score.append("Error")
            audience_score.append("Error")
            time.sleep(random.uniform(5,10))
            #print(genre_count)
            genre_count+=1
            continue

        #print(html.find("404 - Not Found") != -1)
        
        if not html.find("404 - Not Found") != -1:

            soup = BeautifulSoup(html, "html.parser")

            #tv shows
            critic = soup.find_all("div", class_="mop-ratings-wrap__half critic-score")
            audience = soup.find_all("div", class_="mop-ratings-wrap__half audience-score")
            season = soup.find_all("section", class_="panel panel-rt panel-box movie_info")
            series = soup.find_all("div", class_="col-right col-full-xs pull-right")

            if row["type"] == "TV Show":
                
                try:
                    if(len(critic[0].find_all("span", class_="mop-ratings-wrap__percentage"))!=0):
                        tomato_score.append(critic[0].find_all("span", class_="mop-ratings-wrap__percentage")[0].text.replace("\n","").replace(" ","").replace("%",""))
                    else:
                        tomato_score.append("undefined")
                except:
                    tomato_score.append("undefined")
                    
                try:
                    if(len(audience[0].find_all("span", class_="mop-ratings-wrap__percentage"))):
                        audience_score.append(audience[0].find_all("span", class_="mop-ratings-wrap__percentage")[0].text.replace("\n","").replace(" ","").replace("%",""))                
                    else:
                        audience_score.append("undefined")
                except:
                    audience_score.append("undefined")

                if (len(season)!=0):
                    another = season[0].find_all("ul", class_="content-meta info")[0].find_all("li", class_="meta-row clearfix")
                    for a in another:
                        if "Genre" in a.getText():
                            sgenre = a.getText().strip().split("\n")[-1]
                            genre.append(sgenre)
                            #print(genre_count)
                            genre_count+=1
                            break
                elif (len(series)!=0):
                    tr_list = series[0].find_all("tr")
                    if(len(tr_list)!=0):
                        for tr in tr_list:
                            if "Genre" in tr.get_text():
                                sgenre = tr.get_text().strip().split("\n")[-1]
                                genre.append(sgenre)
                                #print(genre_count)
                                genre_count+=1
                                break
                    else:
                        genre.append("undefined")
                        #print(genre_count)
                        genre_count+=1
                else:
                    genre.append("undefined")
                    #print(genre_count)
                    genre_count+=1
            
            #movies
            movie_soup = soup.find("score-board", class_="scoreboard")
            movie_genre = soup.find("div", class_="meta-value genre")

            

            if row["type"] == "Movie":

                if movie_soup is None:
                    tomato_score.append("undefined")
                    audience_score.append("undefined")
                    genre.append("undefined")
                    #print(genre_count)
                    genre_count+=1
                    continue

                try:

                    if not movie_soup.get_attribute_list("audiencescore")[0] == "":
                        audience_score.append(movie_soup.get_attribute_list("audiencescore")[0])
                    else:
                        audience_score.append("undefined")
                
                except:
                    audience_score.append("undefined")

                try:

                    if not movie_soup.get_attribute_list("tomatometerscore")[0] == "":
                        tomato_score.append(movie_soup.get_attribute_list("tomatometerscore")[0])
                    else:
                        tomato_score.append("undefined")
                
                except:
                    tomato_score.append("undefined")

                if not movie_genre is None:
                    genre.append(movie_genre.get_text().replace("\n","").replace("  ","").strip())
                    #print(genre_count)
                    genre_count+=1
                else:
                    genre.append("undefined")
                    #print(genre_count)
                    genre_count+=1
        else:
            tomato_score.append("Not found on RT")
            audience_score.append("Not found on RT")
            genre.append("Not found on RT")
            #print(genre_count)
            genre_count+=1
    else:
        tomato_score.append("Not found on RT")
        audience_score.append("Not found on RT")
        genre.append("Not found on RT")
        #print(genre_count)
        genre_count+=1

0 https://www.rottentomatoes.com/m/dick_johnson_is_dead
0
1 https://www.rottentomatoes.com/tv/blood_and_water_2015
1
2 https://www.rottentomatoes.com/tv/ganglands/s01
2
3 https://www.rottentomatoes.com/tv/jailbirds_new_orleans/s01
3
4 https://www.rottentomatoes.com/tv/kota_factory
4
5 https://www.rottentomatoes.com/tv/midnight_mass/s01
5
6 https://www.rottentomatoes.com/m/my_little_pony_a_new_generation
6
7 https://www.rottentomatoes.com/m/sankofa
7
8 https://www.rottentomatoes.com/tv/the_great_british_baking_show
8
9 https://www.rottentomatoes.com/m/the_starling
9
10 https://www.rottentomatoes.com/tv/vendetta_truth_lies_and_the_mafia/s01
10
11 https://www.rottentomatoes.com/tv/bangkok_breaking/s01
11
12 https://www.rottentomatoes.com/m/je_suis_karl
12
13 https://www.rottentomatoes.com/m/confessions_of_an_invisible_girl
13
14 https://www.rottentomatoes.com/tv/crime_stories_india_detectives/s01
14
15 https://www.rottentomatoes.com/tv/dear_white_people
15
16 Not found on RT
16
17 Not fou

**6. Write final df to file**

In [76]:
netflix["audience_score"] = audience_score
netflix["tomato_score"] = tomato_score

netflix["audience_score"] = netflix["audience_score"].replace("undefined","-1",regex=True).replace("Error", "-1", regex=True).replace("Not found on RT","-2",regex=True)
netflix["tomato_score"] = netflix["tomato_score"].replace("undefined","-1",regex=True).replace("Error", "-1", regex=True).replace("Not found on RT","-2",regex=True)

netflix
netflix.to_csv("final_netflix.csv")